In [1]:
import os
os.chdir("../")

In [2]:
import dagshub
dagshub.init(repo_owner='BhavyaLikhitha', repo_name='Chest-Cancer-Classification-using-MLFlow-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as BhavyaLikhitha

Initialized MLflow to track repo "BhavyaLikhitha/Chest-Cancer-Classification-using-MLFlow-DVC"

Repository BhavyaLikhitha/Chest-Cancer-Classification-using-MLFlow-DVC initialized!

In [3]:
# MLFLOW_TRACKING_URI=https://dagshub.com/BhavyaLikhitha/Chest-Cancer-Classification-using-MLFlow-DVC.mlflow
# MLFLOW_TRACKING_USERNAME=BhavyaLikhitha \
# MLFLOW_TRACKING_PASSWORD =67e311e47e6329264788b16fb4d2dc2a3c432f6b \
# python script.py

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/BhavyaLikhitha/Chest-Cancer-Classification-using-MLFlow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="BhavyaLikhitha"
os.environ["MLFLOW_TRACKING_PASSWORD"]="67e311e47e6329264788b16fb4d2dc2a3c432f6b"

In [3]:
import tensorflow as tf

In [4]:

print(os.path.exists("artifacts/training/model.h5"))


True


In [5]:
model_path = os.path.abspath("artifacts/training/model.h5")
model = tf.keras.models.load_model(model_path)

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from ChestCancerClassification.constants import *
from ChestCancerClassification.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/BhavyaLikhitha/Chest-Cancer-Classification-using-MLFlow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
import mlflow.tensorflow
from urllib.parse import urlparse

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    # def log_into_mlflow(self):
    #     mlflow.set_registry_uri(self.config.mlflow_uri)
    #     tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
    #     with mlflow.start_run():
    #         mlflow.log_params(self.config.all_params)
    #         mlflow.log_metrics(
    #             {"loss": self.score[0], "accuracy": self.score[1]}
    #         )
    #         # Model registry does not work with file store
    #         if tracking_url_type_store != "file":
    #               # Register the model
    #             # There are other ways to use the Model Registry, which depends on the use case,
    #             # please refer to the doc for more information:
    #             # https://mlflow.org/docs/latest/model-registry.html#api-workflow
    #             mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
    #             # mlflow.tensorflow.log_model(self.model, artifact_path="model", registered_model_name="VGG16Model")

    #         else:
    #             mlflow.keras.log_model(self.model, "model")
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
        
        # Model registry does not work with file store
            if tracking_url_type_store != "file":
            # Use mlflow.tensorflow as a fallback if keras logging fails
                mlflow.tensorflow.log_model(
                    self.model,
                    artifact_path="model",
                    registered_model_name="VGG16Model"
            )
            else:
                mlflow.keras.log_model(self.model, "model")


In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-01-01 11:36:11,392: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-01 11:36:11,421: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-01 11:36:11,421: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 37s 5s/step - loss: 22.4961 - accuracy: 0.5686
[2025-01-01 11:36:49,754: INFO: common: json file saved at: scores.json]


2025/01/01 11:36:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-01-01 11:36:53,347: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\bhavy\AppData\Local\Temp\tmpc7igmy44\model\data\model\assets
[2025-01-01 11:36:54,180: INFO: builder_impl: Assets written to: C:\Users\bhavy\AppData\Local\Temp\tmpc7igmy44\model\data\model\assets]


c:\Users\bhavy\.conda\envs\cancer\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/01/01 11:37:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.


In [17]:
import mlflow.tensorflow
import tensorflow as tf

# Dummy model
model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(3,))])
model.compile(optimizer="adam", loss="mse")

with mlflow.start_run():
    mlflow.tensorflow.log_model(model, artifact_path="test_model")


2024/12/31 21:55:04 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bhavy\AppData\Local\Temp\tmprhfgxtgd\model\data\model\assets
[2024-12-31 21:55:04,366: INFO: builder_impl: Assets written to: C:\Users\bhavy\AppData\Local\Temp\tmprhfgxtgd\model\data\model\assets]


2024/12/31 21:55:25 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bhavy\AppData\Local\Temp\tmprhfgxtgd\model, flavor: tensorflow), fall back to return ['tensorflow==2.13.0']. Set logging level to DEBUG to see the full traceback.
c:\Users\bhavy\.conda\envs\cancer\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [13]:
import tensorflow as tf
print(tf.version.VERSION)

2.13.0


In [12]:
import tensorflow.keras
print(hasattr(tensorflow.keras, '__version__'))


True


In [1]:
import tensorflow as tf
print(tf.__version__)  # Should print 2.11.1


2.11.1


In [16]:
pip install tensorflow==2.11.1


In [16]:
import tensorflow.keras
tensorflow.keras.__version__ = tf.__version__


In [12]:
import tensorflow as tf
import mlflow
print("TensorFlow version:", tf.__version__)
print("MLflow version:", mlflow.__version__)


TensorFlow version: 2.13.0
MLflow version: 2.2.2


In [12]:
import distutils
print(distutils.__file__)


c:\Users\bhavy\.conda\envs\cancer\lib\distutils\__init__.py


In [13]:
import setuptools
import distutils
print(setuptools.__version__)
print(distutils.__file__)


75.1.0
c:\Users\bhavy\.conda\envs\cancer\lib\distutils\__init__.py


In [14]:
import setuptools
import distutils
print(f"setuptools version: {setuptools.__version__}")
print(f"distutils location: {distutils.__file__}")


setuptools version: 75.1.0
distutils location: c:\Users\bhavy\.conda\envs\cancer\lib\distutils\__init__.py
